### Match ECHO videos with PET CFR labels ###
Master PET-ECHO match list: All data available on FS. Need to be filtered to create data sets.

In [1]:
import os
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 2000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200425'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200425


#### PET DATA ####

In [3]:
pet_echo_file = os.path.normpath('/mnt/obi0/phi/pet/pet_cfr/pets_with_echos_all.parquet')
pet_cols = ['mrn', 'pet_date', 'petmrn_identifier']
pet = pd.read_parquet(pet_echo_file)
pet = pet[pet_cols].dropna(how='all').drop_duplicates()
pet = pet.astype({'pet_date': 'datetime64[ns]'})
print(f'Unique PET measurements {len(pet.petmrn_identifier.unique())}')
pet.head()

Unique PET measurements 4136


,mrn,pet_date,petmrn_identifier
0,7924277,2008-08-15,7924277_2008-08-15
1,12853099,2006-01-25,12853099_2006-01-25
2,20710471,2006-01-23,20710471_2006-01-23
3,12627030,2006-02-10,12627030_2006-02-10
4,15324312,2006-02-23,15324312_2006-02-23


#### PET ECHO PAIRS (Rahul's list) ####

In [4]:
match_list_file = 'bwh_2020-Feb-4_pet_365_diff.txt'
rdf = pd.read_csv(os.path.join(cfr_data_root, match_list_file), sep = '\t')
rdf = rdf.astype({'echo_date': 'datetime64[ns]',
                  'pet_date': 'datetime64[ns]'}).\
        rename(columns={'MRN': 'mrn'})
rdf.head()

,mrn,echo_date,pet_date,difference(days)
0,17666439,2017-03-14,2017-07-20,128
1,12287447,2019-05-16,2020-01-23,252
2,12287447,2020-01-23,2020-01-23,0
3,31953938,2017-11-09,2018-08-20,284
4,31953938,2018-08-13,2018-08-20,7


In [5]:
# Join PET and PET-ECHO PAIRS tables
pet_echo = rdf.merge(pet, on=['mrn', 'pet_date'], how='left')

In [6]:
# Those rows that do not have the petmrn_identifier did not have a match mrn_pet_date with Rahul's list
pet_echo_noID = pet_echo.loc[pet_echo.petmrn_identifier.isnull()]
pet_echo_noID.head()
print(f'mrn-pet_date pairs without echo_date: {pet_echo_noID.shape[0]}')
# Let's save those
filename = 'missingPET.parquet'
pet_echo_noID.to_parquet(os.path.join(meta_dir, filename))
# Let's focus on those mrn-pet_date pairs that were identified by Rahul
pet2 = pet_echo.loc[~pet_echo.petmrn_identifier.isnull()]
print(f'mrn-pet_dates with echo_dates: {pet2.shape[0]}')
print(f'PET studies {len(pet2.petmrn_identifier.unique())}')

mrn-pet_date pairs without echo_date: 196
mrn-pet_dates with echo_dates: 7227
PET studies 3791


#### NPY ECHO FILES ####
This is a data frame that contains the filenames, MRNs and dates of all echo studies

In [7]:
echo_files_name = 'echo_npyFiles_BWH_'+cfr_meta_date+'.parquet'
echo = pd.read_parquet(os.path.join(meta_dir, echo_files_name))
echo = echo.assign(echo_study_date = echo.datetime.dt.date,
                   echo_study_time = echo.datetime.dt.time)

echo = echo.astype({'mrn': 'int64',
                    'echo_study_date': 'datetime64[ns]'})

print(echo_files_name)
print('Number of npy files: {}'.format(len(echo.filename.unique())))
print('Unique echo studies: {}'.format(len(echo.study.unique())))
print('Unique patients:     {}'.format(len(echo.mrn.unique())))
print()

# For matching dates with PET, we only need the studies
# So we can greatly reduce this data frame
echo = echo[['study', 'mrn', 'echo_study_date']].drop_duplicates().reset_index(drop = True).\
    sort_values(by = 'mrn', ascending = True)

echo_npyFiles_BWH_200425.parquet
Number of npy files: 1390741
Unique echo studies: 27746
Unique patients:     16579



In [8]:
echo.head()

,study,mrn,echo_study_date
3956,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,9241,2010-04-02
3951,4b7b463ae45ad0e0_4903a585842f436837ee2e0d9dd5,9241,2010-01-26
4040,4b7b463e0550e9de_4903a58051d59d426fe452739a29,41723,2015-11-03
3962,4b7b463fc2f49a36_4903a58dd5465407d53d20ba3790,51599,2018-05-22
3971,4b7b463fc2f49a36_4903a58793b43bb4bb1078d62f4e,51599,2012-03-08


#### PET STUDIES + NPY ECHO DATA merged on mrn only ####

In [11]:
pet_study = pet2.merge(right = echo, on = 'mrn', how = 'left')

In [12]:
mrns_without_echo = pet_study.loc[pet_study.study.isnull()]
mrns_without_echo = mrns_without_echo.drop(axis=1, columns=['study', 'echo_study_date']).drop_duplicates()
print(f'mrns with PET total:       {len(pet2.mrn.unique())}')
print(f'mrns with PET but no echo: {len(mrns_without_echo.mrn.unique())}')
mrns_without_echo.head()
filename='MRNs_without_echo.parquet'
mrns_without_echo.to_parquet(os.path.join(meta_dir, filename))

# OK let's drop all rows where we dont have echo studies
pet_study = pet_study.loc[~pet_study.study.isnull()]
print(f'mrns with PET and echo:    {len(pet_study.mrn.unique())}')

mrns with PET total:       3498
mrns with PET but no echo: 618
mrns with PET and echo:    2880


In [13]:
# Now we need to match pet_date and echo_dates (we have two echo dates from the merge with .npy files)
pet_study[pet_study.mrn==39835152]

,mrn,echo_date,pet_date,difference(days),petmrn_identifier,study,echo_study_date
1151,39835152,2019-06-14,2019-09-10,88,39835152_2019-09-10,48bca16ebb9f79e9_4903a58c1bc8b8b4edc2ff652898,2019-07-29
1152,39835152,2019-06-14,2019-09-10,88,39835152_2019-09-10,48bca16ebb9f79e9_4903a58c1bc99380baa33591b299,2019-06-14
1153,39835152,2019-07-29,2019-09-10,43,39835152_2019-09-10,48bca16ebb9f79e9_4903a58c1bc8b8b4edc2ff652898,2019-07-29
1154,39835152,2019-07-29,2019-09-10,43,39835152_2019-09-10,48bca16ebb9f79e9_4903a58c1bc99380baa33591b299,2019-06-14


### Match ECHO study with PET reportID  ###
Goal:
Find a PET study for each echo video taken from the closest PET measurement.
Requirement: We want to pick the most recent CFR measurments for each echo that is not older than 365 days.
For this, we need a data frame that contains for a given MRN:

- All echo studies for this patient
- A date for each echo study
- All pet reports for that patient
- Date for each pet report

In [14]:
df = pet_study.copy()
mrn=39835152
df_mrn = df[df.mrn == mrn]
echo_study_list = list(df_mrn.study.unique())
study = echo_study_list[0]
echo_study_date = df_mrn[df_mrn.study == study].echo_study_date.values[0]
pet_dates = df_mrn[['petmrn_identifier', 'pet_date']].groupby('petmrn_identifier').first().reset_index(drop=False)
pet_dates = df_mrn[['petmrn_identifier', 'pet_date']].groupby('petmrn_identifier').first().reset_index(drop=False)
pet_dates = pet_dates.assign(days_post_pet=(echo_study_date-pet_dates.pet_date).dt.days)
pet_dates = pet_dates.assign(days_abs = np.abs(pet_dates.days_post_pet))
ser = pet_dates.loc[pet_dates.days_abs.idxmin()]

In [15]:
pet_dates

,petmrn_identifier,pet_date,days_post_pet,days_abs
0,39835152_2019-09-10,2019-09-10,-43,43


In [16]:
def match_pet_echo(df):
    """ 
    Determine the earliest PET measurement for all studies
    """

    # Loop through all MRNs in df_pet_echo
    mrn_list = sorted(list(df.mrn.unique()))
    cfr_echo_days = pd.DataFrame()
    for m, mrn in enumerate(mrn_list):

        if (m+1)%200==0:
            print('Processing MRN {} of {}'.format(m+1, len(mrn_list)))

        df_mrn = df[df.mrn == mrn]
        echo_study_list = list(df_mrn.study.unique())

        # Loop through all echo studies for this MRN
        for study in echo_study_list:
            echo_study_date = df_mrn[df_mrn.study == study].echo_study_date.values[0]

            # Find all pet dates for this patient and add the difference to the echo dates
            pet_dates = df_mrn[['petmrn_identifier', 'pet_date']].groupby('petmrn_identifier').first().reset_index(drop=False)
            pet_dates = pet_dates.assign(days_post_pet=(echo_study_date-pet_dates.pet_date).dt.days)
            pet_dates = pet_dates.assign(days_abs = np.abs(pet_dates.days_post_pet))
            ser = pet_dates.loc[pet_dates.days_abs.idxmin()]

            # We can return the reportID and the days of the earliest pet study
            study_dict = {'mrn': [mrn],
                          'study': [study],
                          'pet_date': [ser.pet_date],
                          'echo_date': [echo_study_date],
                          'petmrn_identifier': [ser.petmrn_identifier],
                          'days_post_pet': [ser.days_post_pet]}
            cfr_echo_days = pd.concat([cfr_echo_days, pd.DataFrame(study_dict)], ignore_index=True)
    
    return cfr_echo_days

In [17]:
pet_echo_days = match_pet_echo(pet_study)

Processing MRN 200 of 2880
Processing MRN 400 of 2880
Processing MRN 600 of 2880
Processing MRN 800 of 2880
Processing MRN 1000 of 2880
Processing MRN 1200 of 2880
Processing MRN 1400 of 2880
Processing MRN 1600 of 2880
Processing MRN 1800 of 2880
Processing MRN 2000 of 2880
Processing MRN 2200 of 2880
Processing MRN 2400 of 2880
Processing MRN 2600 of 2880
Processing MRN 2800 of 2880


Here, we have picked the closest PET date for all echo studies for this mrn. And the closest PET date was the same for all echos. Therefore, PET scans that occurred earlier that do not have an echo that is closer, will disappear.

In [18]:
pet_echo_days[pet_echo_days.mrn==mrn]

,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet
5566,39835152,48bca16ebb9f79e9_4903a58c1bc8b8b4edc2ff652898,2019-09-10,2019-07-29,39835152_2019-09-10,-43
5567,39835152,48bca16ebb9f79e9_4903a58c1bc99380baa33591b299,2019-09-10,2019-06-14,39835152_2019-09-10,-88


In [19]:
pet2.head()

,mrn,echo_date,pet_date,difference(days),petmrn_identifier
0,17666439,2017-03-14,2017-07-20,128,17666439_2017-07-20
1,12287447,2019-05-16,2020-01-23,252,12287447_2020-01-23
2,12287447,2020-01-23,2020-01-23,0,12287447_2020-01-23
3,31953938,2017-11-09,2018-08-20,284,31953938_2018-08-20
4,31953938,2018-08-13,2018-08-20,7,31953938_2018-08-20


In [20]:
# I still want to know how this match compares to Rahu's original match list
pet_echo_matched = pet_echo_days.merge(pet2, on = ['mrn', 'echo_date', 'pet_date', 'petmrn_identifier'], how='left')
pet_echo_matched = pet_echo_matched.sort_values(by='mrn').reset_index(drop=True)
pet_echo_matched.head(20)
print(f'PET studies before match with echo: {len(pet2.petmrn_identifier.unique())}')
print(f'PET studies after match with echo:  {len(pet_echo_matched.petmrn_identifier.unique())}')
# After match there are fewer PET studies because we only picked the closest PET measurment for each echo
print(f'mrns in original PET file:          {len(pet.mrn.unique())}')
print(f'mrns in original match file:        {len(pet2.mrn.unique())}')
print(f'mrns with PET and echo after match: {len(pet_echo_matched.mrn.unique())}')

PET studies before match with echo: 3791
PET studies after match with echo:  3001
mrns in original PET file:          3516
mrns in original match file:        3498
mrns with PET and echo after match: 2880


In [21]:
print(f'PET studies in originial match list: {len(pet2.petmrn_identifier.unique())}')
print(f'PET studies in new match list:       {len(pet_echo_matched.petmrn_identifier.unique())}')
print(f'Total ECHO studies in match list:    {len(pet_echo_matched.study.unique())}')

PET studies in originial match list: 3791
PET studies in new match list:       3001
Total ECHO studies in match list:    5574


In [22]:
# Filter echo studies that do not have a PET within 365 days
pet_echo_matched_365 = pet_echo_matched.assign(abs_pet_days = np.abs(pet_echo_matched.days_post_pet))
pet_echo_matched_365 = pet_echo_matched_365[pet_echo_matched_365.abs_pet_days<=365].drop(columns = ['abs_pet_days'])
pet_echo_matched_365 = pet_echo_matched_365.sort_values(by = 'study')
pet_echo_matched_365.head()

,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet,difference(days)
5452,35169325,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,2017-05-22,2017-06-21,35169325_2017-05-22,30,30.0
5450,35133560,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,2018-04-25,2018-11-02,35133560_2018-04-25,191,191.0
5446,35014216,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,2018-05-08,2019-04-25,35014216_2018-05-08,352,352.0
5448,35034941,48b091b6184a771a_4903a582ec751999e07cf30ae81a,2017-04-17,2017-04-18,35034941_2017-04-17,1,1.0
5447,35029545,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,2017-07-24,2017-05-16,35029545_2017-07-24,-69,69.0


In [23]:
# After the match, do we have the same number of patients
print(f'mrns before 365 filter:        {len(pet_echo_matched.mrn.unique())}')
print(f'mrns after 365 filter:         {len(pet_echo_matched_365.mrn.unique())}')
print()
print(f'echo studies before 365 filter:{len(pet_echo_matched.study.unique())}')
print(f'echo studies after 365 filter: {len(pet_echo_matched_365.study.unique())}')

mrns before 365 filter:        2880
mrns after 365 filter:         2719

echo studies before 365 filter:5574
echo studies after 365 filter: 4143


In [24]:
pet_echo_matched_365.head(10)

,mrn,study,pet_date,echo_date,petmrn_identifier,days_post_pet,difference(days)
5452,35169325,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,2017-05-22,2017-06-21,35169325_2017-05-22,30,30.0
5450,35133560,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,2018-04-25,2018-11-02,35133560_2018-04-25,191,191.0
5446,35014216,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,2018-05-08,2019-04-25,35014216_2018-05-08,352,352.0
5448,35034941,48b091b6184a771a_4903a582ec751999e07cf30ae81a,2017-04-17,2017-04-18,35034941_2017-04-17,1,1.0
5447,35029545,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,2017-07-24,2017-05-16,35029545_2017-07-24,-69,69.0
5449,35092402,48b091bc02fb15f5_4903a58dd541f0af3fb9763595d6,2018-01-29,2018-02-12,35092402_2018-01-29,14,14.0
5457,35583509,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,2018-08-08,2017-09-06,35583509_2018-08-08,-336,336.0
5455,35401116,48b095533cbfdf01_4903a58c1bcb2198682a98b7503a,2019-04-11,2019-04-09,35401116_2019-04-11,-2,2.0
5456,35401116,48b095533cbfdf01_4903a58dd5441181fcd696f7d50c,2019-04-11,2018-07-25,35401116_2019-04-11,-260,260.0
5461,35750744,48b09627edb790dc_4903a582ec78ec0420a169da26a4,2017-08-16,2017-09-01,35750744_2017-08-16,16,16.0


In [25]:
# We can now save the data frame with the echo studies and the matched CFR measurements
match_filename = 'pet_match365_diff_'+cfr_meta_date+'.parquet'
pet_echo_matched_365.to_parquet(os.path.join(meta_dir, match_filename))

# Save also as .csv file
match_filename_csv = 'pet_match365_diff_'+cfr_meta_date+'.csv'
pet_echo_matched_365.to_csv(os.path.join(meta_dir, match_filename_csv))
print(pet_echo_matched_365.shape)

(4143, 7)
